In [1]:
!pip install pyodbc # ODBC的DRIVER(驅動程式)

Active code page: 936


ERROR: Invalid requirement: '#'


In [2]:
!pip install selenium

Active code page: 936


In [3]:
# coding=utf-8
from platform import python_version
import os, time, socket, glob, pickle, joblib
import pandas as pd
import pyodbc, pymssql
import warnings
warnings.filterwarnings(action="ignore")
print("【現在時間】{}".format(time.strftime("%Y/%m/%d %H:%M:%S")))
print("【所在目錄】{}".format(os.getcwd()))
#print("【主機名稱】{} ({})".format(socket.gethostname(),socket.gethostbyname(socket.
print("【Python】{}".format(python_version()))
print("【pymssql】{}".format(pymssql.__version__))
print("【pyodbc】{}".format(pyodbc.version))
import selenium
from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import *

%autosave 120

【現在時間】2022/12/18 17:18:05
【所在目錄】C:\Users\User\Desktop\資料工程
【Python】3.9.13
【pymssql】2.2.7
【pyodbc】4.0.35


Autosaving every 120 seconds


In [4]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup

## 注意：每個公司的現金流量表不同 需確定格子狀態

In [9]:
def cash(stockcode):
    url=f"https://goodinfo.tw/tw/StockFinDetail.asp?RPT_CAT=CF_M_QUAR_ACC&STOCK_ID={stockcode}"   #現金流量

    chrome=webdriver.Chrome(executable_path=r'C:\Users\User\NTUproject\mylinebot00\chromedriver.exe')

    chrome.get(url)
    time.sleep(2)
    #chrome.find_element_by_css_selector("#RPT_CAT > option:nth-child(1)").click()
    time.sleep(2)
    result=pd.read_html(chrome.page_source)

    data=pd.DataFrame(result[15]).T
    chrome.quit()
    
    return data

In [24]:
stock_id = "6446"  #1605 1218 6446
table_s=cash(stock_id)
df = table_s.dropna(axis='columns')
df.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
營業活動,本期淨利(淨損),繼續營業單位淨利(淨損),折舊費用,攤銷費用,利息費用提列(迴轉)數,利息收入,股份基礎給付酬勞成本,處分及報廢固定資產損失(利益),處分無形資產損失(利益),其他項目,...,庫藏股之(買回)賣出,員工執行認股權,收取(支付)之利息,融資活動之淨現金流入(出),其他影響現金流量之活動,匯率變動影響數,現金流量總計,本期現金及約當現金增加(減少)數,期初現金及約當現金餘額,期末現金及約當現金餘額
2022Q3,-4.09,-4.09,0.48,0.059,0.029,-0.058,1.7,0,0.001,0,...,0,0.45,-0.007,0.1,2022Q3,0.2,2022Q3,-0.7,42.1,41.4
2022Q2,-2.75,-2.75,0.5,0.06,0.028,-0.027,0.16,0,-,-,...,0,0.14,-0.006,18.18,2022Q2,0.072,2022Q2,15.75,26.35,42.1
2022Q1,-5.41,-5.41,0.6,0.059,0.023,-0.014,0.16,0,-,-,...,0,0.14,-0.005,-0.18,2022Q1,-0.075,2022Q1,-8.19,34.54,26.35
2021Q4,-8.87,-8.87,0.58,0.026,0.024,0.033,4.76,0,0,0,...,0,3.08,-0.006,29.94,2021Q4,-0.008,2021Q4,24.3,10.23,34.54


In [27]:
products_list = df.values.tolist()
print(products_list[10])
'''
del products_list[19]
del products_list[18]'''
lists_num=len(products_list)
print(len(products_list))
print(products_list[1][56])
"0 1   4 5 10   20"

['-2.07', '-2.07', '0.42', '0.01', '0.018', '-0.01', '0.021', '0', '-', '-', '0.46', '0', '-1.14', '-0.053', '-0.65', '-', '0.29', '0.078', '-1.48', '0.025', '0.25', '-0.002', '0.38', '0', '-0.001', '0.058', '0', '0.72', '-0.76', '-0.3', '-2.37', '0.012', '0', '-2.36', '2020Q2', '-', '-', '-0.093', '0', '-0.017', '-0.014', '-0.15', '-0.092', '-0.031', '-0.4', '2020Q2', '0', '-0.014', '-0.19', '15.69', '0', '0.028', '-0.004', '15.5', '2020Q2', '-0.008', '2020Q2', '12.74', '11.63', '24.37']
11
2022Q3


'0 1   4 5 10   20'

In [16]:
connsqlodbc= pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};SERVER=124.218.194.84;"
                      "DATABASE=testDB;UID=testuser;PWD=323")


In [17]:
connsqlodbc.autocommit = True
cursor = connsqlodbc.cursor()
# GS+代號+發周別
# MYSQL指令
sqlStr ='''create table 現金流量表
(
	現金流量表編號 nvarchar(20) primary key
    ,[當期現金流量] nvarchar(20) Null
	,當期現金流量季度 nvarchar(10) not Null
	,股票代號 nvarchar(10) not Null
)
'''
cursor.execute(sqlStr)

In [28]:
connsqlodbc.autocommit = True

connsqlodbc.setdecoding(pyodbc.SQL_CHAR, encoding='utf-8')
connsqlodbc.setencoding('utf-8')
cursor = connsqlodbc.cursor()
# CA+代號+發周別
# MYSQL指令
for i in range(0,lists_num):
    CAstock_id='CA'+str(stock_id)+str(products_list[i][56])
    sqlStr = f""" 
   Insert into 現金流量表 (現金流量表編號
    ,[當期現金流量] 
	,當期現金流量季度
	,股票代號)
    values ('{str(CAstock_id)}',
    '{products_list[i][57]}',
    '{products_list[i][56]}','{stock_id}')
    
    """
    cursor.execute(sqlStr)

print(sqlStr)



 
   Insert into 現金流量表 (現金流量表編號
    ,[當期現金流量] 
	,當期現金流量季度
	,股票代號)
    values ('CA64462020Q2',
    '12.74',
    '2020Q2','6446')
    
    
